#Tema 2 IA
###Studenti: 
###Sg. Maj. Badea Mihai-Valentin
###Sg. Maj. Pesu Ciprian-George
###Sg. Maj. Nancu Petrica
###Neacsu Stefan

In [60]:
import pickle
import pandas
import json
import spacy
import numpy
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
from matplotlib import pyplot as plt
import random
import string
import unicodedata

##Transcriere dataset in DataFrame pandas

In [6]:
df=pandas.read_pickle("/content/drive/MyDrive/dataset.pickle")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
dictionaries = []
for i in df.keys():
    examples = json.dumps(df[i])
    dictionaries.append(json.loads(examples))

In [8]:
dictionaries[0][1]

{'correct_synset_id': 'ENG30-08199025-n',
 'literal': 'armată',
 'sentence': 'CERD CA VA CONDUCE ARMATA SA LA VICTORIE IMOPTRIVA CELOR CARE NE-AU RAPIT LIBERATEA.',
 'synsets': 'ENG30-08183290-n ENG30-08198398-n ENG30-08199025-n ENG30-08191230-n -1 ',
 'text': 'ARMATA',
 'text_postfix': ' SA LA VICTORIE IMOPTRIVA CELOR CARE NE-AU RAPIT LIBERATEA.',
 'text_prefix': 'CERD CA VA CONDUCE ',
 'user_id': 'Novac Andrei'}

In [9]:
max_synset=0
max_sentence=0

vect_literal=[]
vect_sentence=[]
vect_correct_synset_id=[]
vect_synsets=[]
vect_cuvantul_target=[]
for i in range(0, len(dictionaries)):
    for j in range(len(dictionaries[i])):
        vect_literal.append(dictionaries[i][j]["literal"])
        vect_sentence.append(dictionaries[i][j]["sentence"])
        vect_correct_synset_id.append(dictionaries[i][j]["correct_synset_id"])
        vect_cuvantul_target.append(dictionaries[i][j]["text"])
        vect_synsets.append(dictionaries[i][j]["synsets"])




        if len(dictionaries[i][j]["sentence"]) > max_sentence:
          max_sentence=len(dictionaries[i][j]["sentence"])
        if len(dictionaries[i][j]["synsets"].split()) > max_synset:
          max_synset=len(dictionaries[i][j]["synsets"].split())

print(max_synset)
print(max_sentence)

35
250


In [10]:
data = {'literal': vect_literal ,
        'sentence': vect_sentence,
        'cuvantul_target':vect_cuvantul_target,
        'correct_synset_id':vect_correct_synset_id,
        'synsets':vect_synsets}

df = pandas.DataFrame(data)

In [27]:
df = df[~df['correct_synset_id'].isin(["-1"])] #eliminam propozitii ce au ca synset corect -1

In [36]:
df.head(20)

,literal,sentence,cuvantul_target,correct_synset_id,synsets
0,armată,În calitate de membri ai Forței Speciale de In...,Armatei,ENG30-08191230-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
1,armată,CERD CA VA CONDUCE ARMATA SA LA VICTORIE IMOPT...,ARMATA,ENG30-08199025-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
2,armată,TOT NU MI-AI SPUS DE CE AI PLECAT DIN ARMATA.,ARMATA,ENG30-08198398-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
3,armată,"EI BINE, DACA TOTI VETI MURI, NU VETI MAI AVEA...",ARMATA,ENG30-08199025-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
4,armată,"DOAR IMPREUNA, PUTEM FI O ARMATA DE LUPTATORI.",ARMATA,ENG30-08183290-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
5,armată,S-a hotărât după aceea să se înroleze în armat...,armata,ENG30-08199025-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
6,armată,Din documentele pe care le avem acest cor era ...,armatei,ENG30-08191230-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
7,armată,"Apoi, între anii 1969-1971 și-a satisfăcut sta...",Armata,ENG30-08198398-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
9,armată,"În timpul luptei apare însă Odiseu, în fruntea...",armate,ENG30-08191230-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...
10,armată,"O armată este o forță puternic înarmată, extre...",armată,ENG30-08198398-n,ENG30-08183290-n ENG30-08198398-n ENG30-081990...


##Preprocesare date

In [40]:
propozitii=list(df["sentence"])
sensuri=list(df["synsets"])
corecte=list(df["correct_synset_id"])
cuvinte_target=list(df["cuvantul_target"])
literal=list(df["literal"])
uniq_literal = list(dict.fromkeys(literal))

In [29]:
####stop words
f=open("/content/drive/MyDrive/stopwords.txt","r");
stop_words=f.read()



In [71]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [72]:
def clean(prop): 
  #elimina semnele de punctuatie din propozitie si o imparte in cuvinte lowercase
  token_prop=strip_accents(prop.lower().translate(str.maketrans('', '', string.punctuation))).split()
  clean_token=[]
  for token in token_prop:
    #eliminam stopwords din vectorul de cuvinte
    if token not in stop_words:
      clean_token.append(token)

  prop=""
  #refacem propozitia dupa preprocesare
  for token in clean_token:
    prop+=token+" "
  
  return prop

In [73]:
clean_propozitii=[]
for i  in range(0,len(propozitii)):
  clean_propozitii.append(clean(propozitii[i]))
      

In [74]:
clean_propozitii #vector ce contine toate propozitiile preprocesate


['calitate membri fortei speciale interventie armatei macedonene irak participat operatiune salvare soldatilor americani ',
 'cerd conduce armata victorie imoptriva neau rapit liberatea ',
 'miai spus plecat armata ',
 'veti muri veti armata ',
 'impreuna putem armata luptatori ',
 'hotarat inroleze armata americana ',
 'documentele alcatuit soldati ofiteri subofiteri armatei numele fiind cunoscute 2 referate 3 octombrie 5 decembrie1940 ',
 'anii 19691971 sia satisfacut stagiul militar obligatoriu armata sovietica ',
 'timpul luptei apare odiseu fruntea mari armate grecesti intorc lupta favoarea eroilor devin invingatori ',
 'armata forta puternic inarmata extrem organizata destinata rand razboiului cunoscuta colectiv forte armate ',
 'tarziu dezvaluit afara armata comandantii iau inscenat furtul planuri secrete temanduse treaca partea thanatos tatal ',
 'armata revolutionara ucraineana formata teritoriile ucrainene urma continue lupta armata aliata germana impotriva ocupatiei ruse ava

##Impartirea datelor in test si train

In [76]:
import random
number = random.sample(range(0,len(clean_propozitii)), int(len(clean_propozitii)/10))

test_prop=[]
test_sensuri=[]
test_cuvant=[]
test_corect=[]


train_prop=[]
train_sensuri=[]
train_cuvant=[]
train_corect=[]


for i in range(0,len(clean_propozitii)):
    if i in number:
        test_prop.append(clean_propozitii[i])
        test_sensuri.append(sensuri[i])
        test_cuvant.append(cuvinte_target[i])
        test_corect.append(corecte[i])
    else:
        train_prop.append(clean_propozitii[i])
        train_sensuri.append(sensuri[i])
        train_cuvant.append(cuvinte_target[i])
        train_corect.append(corecte[i])


In [77]:
print(len(test_prop))
print(len(test_sensuri))
print(len(test_cuvant))
print(len(test_corect))
print("---------------")
print(len(train_prop))
print(len(train_sensuri))
print(len(train_cuvant))
print(len(train_corect))

15214
15214
15214
15214
---------------
136933
136933
136933
136933


In [20]:
!pip install rowordnet

     |████████████████████████████████| 11.8 MB 5.2 MB/s 


In [21]:
import rowordnet as rwn
wn = rwn.RoWordNet()
word = 'arbore'
synset_ids = wn.synsets(literal=word)

In [22]:
wn.print_synset("ENG30-08191230-n")

Synset: 
	  id=ENG30-08191230-n
	  pos=NOUN
	  nonlexicalized=None
	  stamp=bozianul
	  domain=military
	  definition=organizarea forțelor militare de uscat ale unei națiuni sau ale unui stat
	  sumo=Army 
	  sumoType=HYPERNYM
	  sentiwn=[0.0, 0.0, 1.0]
	  Literals:
		  armată - 1.2
		  armie - 1
		  oștire - 1
		  oaste - 1
		  trupă - 2.x
	  Outbound relations: 
		  ENG30-08198137-n - hypernym
		  ENG30-08199025-n - domain_TOPIC
		  ENG30-08190754-n - member_meronym
		  ENG30-08212527-n - member_meronym
		  ENG30-08394423-n - hyponym
		  ENG30-08394535-n - hyponym
	  Inbound relations: 
		  ENG30-08199025-n - domain_member_TOPIC
		  ENG30-08190754-n - member_holonym
		  ENG30-08212527-n - member_holonym
		  ENG30-08198137-n - hyponym
		  ENG30-08394423-n - hypernym
		  ENG30-08394535-n - hypernym


##Implementarea algoritmului Lesk

In [23]:
def overlap(sense,context):

  object_wn=wn(sense)
  clean_sense=clean(object_wn.definition)

  nr=0
  for word in clean_sense.split():
    if word in context.split():
        nr+=1
  
  for i in range(0,len(train_prop)):
    if train_corect[i] == sense:
      for word in context.split():
        if word in train_prop[i].split():
          nr+=1


  return nr


In [24]:
def lesk(word,context,sensuri):
  best_sense=""
  max_overlap=0

  for sense in sensuri.split():
    if sense != "-1":
      current_overlap=overlap(sense,context)
      
      if current_overlap>max_overlap:
        max_overlap=current_overlap
        best_sense=sense

  return best_sense      

##Testare algoritm implementat

In [78]:
corect=0
gresit=0
vector_metrica_pozitiv=[0]*len(uniq_literal)
vector_metrica_negativ=[0]*len(uniq_literal)
for i in range(0,len(test_cuvant)):
  aux=lesk(test_cuvant[i],test_prop[i],test_sensuri[i])

  if(aux==test_corect[i]):
    corect+=1
    index=uniq_literal.index(literal[i])
    vector_metrica_pozitiv[index]+=1
  else:
    gresit+=1
    index=uniq_literal.index(literal[i])
    vector_metrica_negativ[index]+=1
  if (i % 100==0) and (i != 0):
    print("Step: "+str(i)+" -> Corecte: " + str(corect) +" Gresite:" + str(gresit) + "\tEficienta: {:.2f}%".format(corect*100/i))

precizie=0
precizie=corect*100/len(test_cuvant)
print("Eficienta totala: {:.2f}%".format(precizie)) 

Step: 100 -> Corecte: 73 Gresite:28	Eficienta: 73.00%
Step: 200 -> Corecte: 117 Gresite:84	Eficienta: 58.50%
Step: 300 -> Corecte: 173 Gresite:128	Eficienta: 57.67%
Step: 400 -> Corecte: 214 Gresite:187	Eficienta: 53.50%
Step: 500 -> Corecte: 274 Gresite:227	Eficienta: 54.80%
Step: 600 -> Corecte: 312 Gresite:289	Eficienta: 52.00%
Step: 700 -> Corecte: 364 Gresite:337	Eficienta: 52.00%
Step: 800 -> Corecte: 405 Gresite:396	Eficienta: 50.62%
Step: 900 -> Corecte: 455 Gresite:446	Eficienta: 50.56%
Step: 1000 -> Corecte: 509 Gresite:492	Eficienta: 50.90%
Step: 1100 -> Corecte: 560 Gresite:541	Eficienta: 50.91%
Step: 1200 -> Corecte: 599 Gresite:602	Eficienta: 49.92%
Step: 1300 -> Corecte: 648 Gresite:653	Eficienta: 49.85%
Step: 1400 -> Corecte: 707 Gresite:694	Eficienta: 50.50%
Step: 1500 -> Corecte: 750 Gresite:751	Eficienta: 50.00%
Step: 1600 -> Corecte: 819 Gresite:782	Eficienta: 51.19%
Step: 1700 -> Corecte: 876 Gresite:825	Eficienta: 51.53%
Step: 1800 -> Corecte: 916 Gresite:885	Efic

In [79]:
total=[0]*len(uniq_literal)
accuracy=[0]*len(uniq_literal)

for i in range(0,len(uniq_literal)):
  total[i]=vector_metrica_negativ[i]+vector_metrica_pozitiv[i]
  if total[i]!=0:
    accuracy[i]=vector_metrica_pozitiv[i]*100/total[i]

accuracy = [ '%.2f' % elem for elem in accuracy ]
data2 = {'Literal': uniq_literal ,
         'Total':total,
         'Corect':vector_metrica_pozitiv,
         'Gresit':vector_metrica_negativ,
         'Acuratete (%)':accuracy}

df2 = pandas.DataFrame(data2)

In [80]:
df2.head(25)

,Literal,Total,Corect,Gresit,Acuratete (%)
0,armată,40,24,16,60.00
1,rol,50,40,10,80.00
2,secol,20,9,11,45.00
3,județ,20,10,10,50.00
4,oraș,50,25,25,50.00
5,persoană,30,16,14,53.33
6,comună,10,3,7,30.00
7,loc,225,119,106,52.89
8,uniune,10,3,7,30.00
9,parte,290,129,161,44.48
